In [66]:
import re 
import requests
import json


class Maoyan(object):
    def __init__(self):
        self.url = "https://maoyan.com/board/4?offset={}"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
            "Cookie": "__mta=44373403.1568707842875.1568707937069.1568707995099.9; _lxsdk_cuid=16d2d2561931-0a31202ebc6845-79475878-3c000-16d2d256194c8; uuid_n_v=v1; uuid=A5005A50D92211E9AFA53B88E9213B3C5B1D8699CC9F4F899C7C340A5AFA8E88; _csrf=5519df7f94e06e9680b6943b869454f49d6321b2949bf4a570f45f49d1351e4c; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; _lxsdk=A5005A50D92211E9AFA53B88E9213B3C5B1D8699CC9F4F899C7C340A5AFA8E88; __mta=44373403.1568707842875.1568707844633.1568707867388.3; _lxsdk_s=16d3e4612f3-d9a-f82-3b2%7C%7C26"
        }
        
    def parse_url(self, url):
        print(url)
        r = requests.get(url, headers=self.headers)
        if r.status_code == 200:
            return r.text
        return None
    
    def get_data(self, html):
        """
        获取电影数据
        返回：电影名， 明星， 发行时间， 链接地址(文件在服务器的相对位置)
        """
        # 获取电影名
        p_film_name = re.compile(r'<p.*?><a.*?films.*?title=\"(.*?)\".*?</a>', re.S)
        film_name = re.findall(p_film_name, html)
        
        # 获取表演明星
        p_star = re.compile(r'<p class="star">\s+(.*?)\s+</p>', re.S)
        star = re.findall(p_star, html)
        
        # 获取上映时间
        p_releasetime = re.compile(r'<p class="releasetime">上映时间：(.*?)</p>')
        releasetime = re.findall(p_releasetime, html)
        
        # 获取链接
        p_href = re.compile(r'<p.*?><a href="(.*?)" title', re.S)
        href = re.findall(p_href, html)
        return film_name, star, releasetime, href
                                 
    def save_data(self, movie_data):
        film_name, star, releasetime, href = movie_data
        movie_info = []
        for i in range(len(film_name)):
            item = {}
            item['film_name'] = film_name[i]
            item['star'] = star[i]
            item["releasetime"] = releasetime[i]
            item['href'] = "https://maoyan.com" + href[i]
            movie_info.append(item)
        with open("../crawl_file/3-3.maoyan_top100.txt", 'a+', encoding='utf-8') as f:

            for i in range(len(film_name)):
                item = {}
                item['film_name'] = film_name[i]
                item['star'] = star[i]
                item["releasetime"] = releasetime[i]
                item['href'] = "https://maoyan.com" + href[i]
                f.write(json.dumps(item, ensure_ascii=False))
                f.write('\n')
        print('保存成功')
                       
            
    def run(self):
        """实现主程序"""
        # 1.设置url 连接页码，并获取url
        current_page = 0
        while True:
            url = self.url.format(current_page)
            # 2.发送请求，获取数据
            html = self.parse_url(url)
            # 3。提取数据
            movie_data = self.get_data(html)
            # 4.保存数据
            self.save_data(movie_data)
            # 设置退出条件
            if (len(movie_data[0]) < 10) or (current_page == 90):
                break
            # 5.构造下一个url
            current_page += 10
        
    
    

In [67]:
if __name__ == "__main__":
    maoyan = Maoyan()
    movie_data = maoyan.run()

https://maoyan.com/board/4?offset=0
保存成功
https://maoyan.com/board/4?offset=10
保存成功
https://maoyan.com/board/4?offset=20
保存成功
https://maoyan.com/board/4?offset=30
保存成功
https://maoyan.com/board/4?offset=40
保存成功
https://maoyan.com/board/4?offset=50
保存成功
https://maoyan.com/board/4?offset=60
保存成功
https://maoyan.com/board/4?offset=70
保存成功
https://maoyan.com/board/4?offset=80
保存成功
https://maoyan.com/board/4?offset=90
保存成功


In [ ]:
import json
import requests
from requests.exceptions import RequestException
import re
import time


def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None


def parse_one_page(html):
    pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?data-src="(.*?)".*?name"><a'
                         + '.*?>(.*?)</a>.*?star">(.*?)</p>.*?releasetime">(.*?)</p>'
                         + '.*?integer">(.*?)</i>.*?fraction">(.*?)</i>.*?</dd>', re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'index': item[0],
            'image': item[1],
            'title': item[2],
            'actor': item[3].strip()[3:],
            'time': item[4].strip()[5:],
            'score': item[5] + item[6]
        }


def write_to_file(content):
    with open('result.txt', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')


def main(offset):
    url = 'http://maoyan.com/board/4?offset=' + str(offset)
    html = get_one_page(url)
    for item in parse_one_page(html):
        print(item)
        write_to_file(item)


if __name__ == '__main__':
    for i in range(10):
        main(offset=i * 10)
        time.sleep(1)
